In [ ]:
## EXAMPLE 3 -- Multiple hypothesis testing
# Consider flipping a coin. There are two outcomes (heads or tails), and so this is called a "Bernoulli trial" (or binomial trial).
# Let's flip the coin 5 times, using the binomial distribution with a probability of 0.5.
# Assume landing on heads is "1" and landing on tails is "0".

Flips <- rbinom(5,1,.5)
cat("Flips:",Flips,"\nNumber of heads:",sum(Flips),"\n\n")


# Now let's simulate this for many people.
# Flip 6000 coins, and split into 10000 people (so same as each flipping 10 times).

Flips <- matrix(rbinom(10000,1,.5),nrow = 1000)

# Sum over each row, so counting how many heads each person flipped.
# Print the mean number of heads each person flipped, and a histogram.

Sums <- rowSums(Flips)
MeanHeads <- mean(Sums)
cat("Mean number of heads flipped:",MeanHeads,"\n")
hist(Sums,breaks = seq(-1,11,length.out = 12))

# How does it look per person?
# Would someone say the coin if unfair if they rolled 2 heads? 9 heads?

cat("Rolling 2 heads: p =",binom.test(2,10,.5)$p.value,"\nRolling 9 heads: p =",binom.test(9,10,.5)$p.value,"\n")

# We have to correct for the number of people "testing" the coin, in this case 100000 people.
# So the critical p-value isn't 0.05 anymore, but 0.05/100000 = 5e-7.

# Let's push it a bit further, consider N flips and B people (so N/B flips per person)

N = 10000
B = 500
cat("Number of flips per person: ",N/B,"\n")

# Change the seed used to generate random numbers, so we can *reproducibly* generate random datasets.
# It is a complicated topic, but don't fall into the trap that similar seeds produce similar results.
# Try commenting one value out and swapping the other in and see what happens.

set.seed(6)
#set.seed(5)

# Generate per-person p values for a binomial test.
# `sapply` applies the function given in the second argument to each element of data in the first argument.
# This gives us the raw p values that each person calculated for their set of N/B flips.

RawP <- sapply(sort(rowSums(matrix(rbinom(N,1,.5),nrow = B))), function(x) binom.test(x, N/B, .5)$p.value)
cat("Number of \"significant\" tests without correction:",sum(RawP<0.05),"\n")               

# Now we can correct the p values using two different methods.
# Note that this adjusts the p values upwards, which is essentially equal to but more obvious than adjusting the significance threshold downwards.

BP <- p.adjust(RawP,"bonferroni")
BHP <- p.adjust(RawP,"BH")
cat("Number of \"significant\" tests with Bonferroni correction:",sum(BP<0.05),"\n")       
cat("Number of \"significant\" tests with Benjamini-Hochberg correction:",sum(BHP<0.05),"\n")       
               
# Plot the raw p values on the x-axis, and the two methods of correction on the y-axis.
# The significance threshold is still p < 0.05, marked by the line.

plot(RawP,BP,col=rgb(0,0,1),ylim=c(0,1),xlab="Raw p",ylab="Adjusted p")
par(new = TRUE)
plot(RawP,BHP,col=rgb(1,0,0),ylim=c(0,1),xlab="",ylab="")
abline(0.05,0)
legend("right",c("Bonferroni","Benjamini-Hochberg"),fill=c(rgb(0,0,1),rgb(1,0,0)))


In [ ]:
## EXAMPLE 1
# Load in data used in the previous lecture

crop_data <- read.table('Frossard_2019.csv',header=TRUE,sep=",")
head(crop_data)

# Let's look at the variance for two of the variables.

var(crop_data$DW.grain.kg.ha)
var(crop_data$SPAD.afternoon)

# Ignoring any assumptions, we can use an F-test to see if they have equal variances.

var.test(crop_data$DW.grain.kg.ha,crop_data$SPAD.afternoon)

# Slightly more advanced, but we can compare the variances between two groups (the two varieties) for specific variables.
# Here, let's look at the "DW.grain.kg.ha" quantity, and see if there is a difference in variance for the two varieties.
# We can do that manually by separating out the data and testing (using the more robust Bartlett test), or a more advanced notation using "~".

bockris <- crop_data[crop_data$variety == "Bockris",]
titlis <- crop_data[crop_data$variety == "Titlis",]
bartlett.test(list(bockris$DW.grain.kg.ha,titlis$DW.grain.kg.ha))

# And this should be equal (but easier to write!)

bartlett.test(DW.grain.kg.ha ~ variety, data = crop_data)

In [ ]:
# EXAMPLE 2

# Load a default dataset for the weight of plants after 3 different types of treatments.
# Plot the weights as boxplots to visualise potential group differences.

boxplot(weight ~ group, data = PlantGrowth, frame = FALSE,
        col = c("#999999", "#E69F00", "#56B4E9"))

# We want to test if there is any significant differences between the three groups.
# We will use the Kruskal-Wallis test.

kruskal.test(PlantGrowth$weight, PlantGrowth$group)


# However, this just shows that there is a significant difference, but it is not clear between which groups.
# We can do pairwise tests (e.g. "ctrl" and "trt1", "ctrl" and "trt2", etc.) to see which are significant.
# Be careful, as we are now testing multiple hypotheses and so should correct accordingly.

In [ ]:
PlantGrowth
pairwise.wilcox.test(PlantGrowth$weight, PlantGrowth$group,   p.adjust.method = "BH",exact=F)
pairwise.wilcox.test(PlantGrowth$weight, PlantGrowth$group,   p.adjust.method = "none",exact=F)

In [ ]:
## EXAMPLE 3

# We can also look at a different dataset.
# Use the "iris" default dataset for flowers.

head(iris)

# Here we will plot the sepal width for the three different species

boxplot(Sepal.Width ~ Species, data = iris, frame = FALSE,
        col = c("#999999", "#E69F00", "#56B4E9"))

# However, there are other variables, so we can try and plot this as an "all-versus-all" pair plot.
# First we should convert the numeric data to a matrix, and also turn the species names into numeric values to make plotting easier.

ma <- as.matrix(iris[, 1:4])
speciesID <- as.numeric(iris$Species)
pairs(ma, col = rainbow(3)[speciesID])

In [ ]:
# Let's test the significance of sepal length for the different species.
kruskal.test(iris$Sepal.Length, iris$Species)

# And again do a pairwise test to see which groups are significant.

pairwise.wilcox.test(iris$Sepal.Length, iris$Species,   p.adjust.method = "BH",exact=F)

In [ ]:
# install a package called matrixTests we will use for simplicity.
#install.packages("matrixTests")

In [ ]:
# Load the package and then do a column-wise test for all the variables

library("matrixTests")
col_kruskalwallis(ma, speciesID)

# Compare to doing each test manually, and confirm identical results.

kruskal.test(iris$Sepal.Length, iris$Species)
kruskal.test(iris$Sepal.Width, iris$Species)
kruskal.test(iris$Petal.Length, iris$Species)
kruskal.test(iris$Petal.Width, iris$Species)

# Note the p-values that are too small get reported as < 2.2e-16, because the computer loses accuracy for numbers smaller than this.
# To get the exact number back, we can do

kruskal.test(iris$Petal.Width, iris$Species)$p.value

In [ ]:
# I couldn't find a simple approach to do multiple multiple hypothesis testing, so we can do this manually.

pairwise.wilcox.test(iris$Sepal.Length, iris$Species,   p.adjust.method = "BH",exact=F)
pairwise.wilcox.test(iris$Sepal.Width, iris$Species,   p.adjust.method = "BH",exact=F)
pairwise.wilcox.test(iris$Petal.Length, iris$Species,   p.adjust.method = "BH",exact=F)
pairwise.wilcox.test(iris$Petal.Width, iris$Species,   p.adjust.method = "BH",exact=F)

In [ ]:
## EXAMPLE 4

# Test if the data is normally distributed (a requirement for anova)

shapiro.test(crop_data$TKG.g)
shapiro.test(crop_data$nr.ears.m2)

# Perform a regression analysis and assess the p-value

summary(lm(TKG.g ~ nr.ears.m2, data = crop_data))

# Jumping ahead a bit, but this is the analysis of variance (ANOVA) approach, which is a type of parametric f-test.

summary(aov(TKG.g ~ nr.ears.m2, data = crop_data))

# Since one of the variables was not normally distributed, we failed the required assumptions.
# Kruskal-Wallis is again the more appropriate test to use, and notice the diffence in p-value.

kruskal.test(TKG.g ~ nr.ears.m2, data = crop_data)

# The order-response variables **matters** a lot.
# If you swap the order, it is no longer the same analysis (as it was in correlation and many other statistical tests).

kruskal.test(nr.ears.m2 ~ TKG.g, data = crop_data)

In [ ]:
# Consider another set of variables, are they normal?

shapiro.test(crop_data$DW.grain.kg.ha)
shapiro.test(crop_data$SPAD.afternoon)

# Go ahead and calculate and plot the regression so we can visualise the relationship.
regression <- lm(DW.grain.kg.ha ~ SPAD.afternoon, data=crop_data)
summary(regression)
plot(DW.grain.kg.ha ~ SPAD.afternoon, data = crop_data)
abline(regression,col="red",lw=4)

# The data does approximately satisfy the requirements for an F-test, so the nonparametric version is less powerful.

kruskal.test(DW.grain.kg.ha ~ SPAD.afternoon, data = crop_data)


# Given a F statistic, how many samples would we need (assuming they follow the same pattern as the existing data) to be significant?

my.F <- 3.883
my.n <- 18:1000
my.p <- pf(my.F, 1, my.n-2, lower.tail=F)
plot(my.p, my.n, type="l")
abline(v=0.05, col="red")

In [ ]:
## Let's look at a more advanced example again

# Read in the semi-colon separated data, skipping the first line which is a comment.
births <- read.csv('cattle-birthCanton.csv',sep=';',skip=1)

# We want to sum over *all* columns by aggregating over year, so we can use the ~ syntax over "." (all coloumns). 
grouped_births <- aggregate(. ~ Year,data=births, FUN=sum)
grouped_births


# We'll look at a subset of four cantons and plot the number of births of the different years
melted_births_subset1 <- melted_births[melted_births$Canton %in% c('AG','VD','BE','VS'),]
plot(Births ~ Year, data = melted_births_subset1,col=factor(melted_births_subset1$Canton),frame = FALSE)

# and we can test for correlations between the cantons, here are two examples.
cor.test( grouped_births$AG, grouped_births$VD)
cor.test( grouped_births$BE, grouped_births$VS)


# We want to test if there is any significant differences between the three groups.
# We will use the Kruskal-Wallis test.

# We'll take another subset of cantons, and reshape the dataframe ("melt" it) so the cantons change from being columns to being row values
melted_births <- reshape2::melt(grouped_births, id.vars = c("Year", "Month"),variable.name = "Canton", value.name = "Births")
melted_births_subset2 <- melted_births[melted_births$Canton %in% c('GR','TG','ZH','SZ','SO','UR','TI'),]

# We need to convert the column type from "fct" (function) to "str" (string), which will remove all the cantons we don't care about above.
# You should try redoing the boxplot after commenting out these two lines and seeing what changes.
i <- sapply(melted_births_subset2, is.factor)
melted_births_subset2[i] <- lapply(melted_births_subset2[i], as.character)


boxplot(Births ~ Canton, data = melted_births_subset2, frame = FALSE)

# We will use the Kruskal-Wallis test to test if there is any significant differences between the cantons.
kruskal.test(Births ~ Canton, data = melted_births_subset2)


# However, this just shows that there is *a* significant difference, but it is not clear between which groups.
# We can do pairwise tests (e.g. SZ and ZH, SZ and SO, etc.) to see which are significant.
# Be careful, as we are now testing multiple hypotheses and so we need to correct accordingly.

pairwise.t.test(melted_births_subset2$Births, melted_births_subset2$Canton, p.adjust.method = "bonf",paired=TRUE)